**Preparing dataframe for analysis:**
- Goal: match the football rankings week-by-week data together by season, week, and team name. This will require (1) standardizing the team names and (2) standardizing the week names.

**Errors**
- Weeky by Week errors:
    - Does Georgia Southern Eagles have a week by week ranking -- they don't appear in the week by week rankings. Seems like quite a few do not have an associated state. (e.g., only 85 teams)
    - Purdue Boilermakers is not in the list
- Negative issues
    - Some weeks have negative values

In [30]:
import pandas as pd
import seaborn as sn

df_football = pd.read_csv('CollegeFootballRankings24to20.csv')
df_rankings = pd.read_csv('WeekByWeekRankings2020to2025.csv')
df_teams = pd.read_csv('2025 Teams.csv')

# Update Week Dataframe
df_rankings_cleaned = pd.merge(df_rankings, df_teams, how="left", on="Team")
print(f"There are {df_rankings_cleaned['team_and_mascot'].isnull().sum()} errors")
df_rankings_cleaned = df_rankings_cleaned.rename(columns={"year": "season", "Team":"state","Week" : "week","team_and_mascot" : "team"}) # Renamed
df_football = df_football.rename(columns={"Team" : "team"})
df_rankings_cleaned.to_csv("WeekbyWeekRankingsUpdated.csv")

# Merge the final Dataframe
df_analyze = pd.merge(df_football, df_rankings_cleaned, how = "left", on=["team","season","week"])
df_analyze.to_csv("Cleaned_Dataset.csv")




There are 2 errors
